In [1]:
import glob
import pandas as pd
import numpy as np


In [2]:
frames = []
for path in glob.glob("../dataset/*.csv"):
    df = pd.read_csv(path, delimiter=";")
    #print(path)
    #print(df.head(2))
    frames.append(df)
data = pd.concat(frames)

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (0,1,2,3,4,5,7,9,10,11,13,14,15,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data.drop('BETRIEBSTAG', axis=1, inplace=True)
data.dropna(axis=0, subset=['PRODUKT_ID'], inplace=True)
data.drop('PRODUKT_ID', axis=1, inplace=True)
data.drop('BETREIBER_ABK', axis=1, inplace=True)
data.drop('BETREIBER_NAME', axis=1, inplace=True)
data.drop('FAHRT_BEZEICHNER', axis=1, inplace=True)
data.drop('BETREIBER_ID', axis=1, inplace=True)
data.drop('UMLAUF_ID', axis=1, inplace=True)
data.drop('VERKEHRSMITTEL_TEXT', axis=1, inplace=True)
data.drop('FAELLT_AUS_TF', axis=1, inplace=True)
data.drop('BPUIC', axis=1, inplace=True)
data.drop('ZUSATZFAHRT_TF', axis=1, inplace=True)
data.dropna(axis=0, inplace=True)

In [4]:
data.shape

(2692052, 10)

In [5]:
data.head(5)

,LINIEN_ID,LINIEN_TEXT,HALTESTELLEN_NAME,ANKUNFTSZEIT,AN_PROGNOSE,AN_PROGNOSE_STATUS,ABFAHRTSZEIT,AB_PROGNOSE,AB_PROGNOSE_STATUS,DURCHFAHRT_TF
17,10057.0,IR,Thun,18.08.2016 13:48,18.08.2016 13:48,GESCHAETZT,18.08.2016 13:49,18.08.2016 13:49,GESCHAETZT,False
18,10057.0,IR,Spiez,18.08.2016 13:59,18.08.2016 13:57,GESCHAETZT,18.08.2016 14:00,18.08.2016 14:00,GESCHAETZT,False
23,101.0,ICE,Wuppertal Hbf,18.08.2016 05:49,18.08.2016 05:52,PROGNOSE,18.08.2016 05:51,18.08.2016 05:54,PROGNOSE,False
24,101.0,ICE,Köln Hbf,18.08.2016 06:47,18.08.2016 06:46,PROGNOSE,18.08.2016 06:55,18.08.2016 06:55,PROGNOSE,False
25,101.0,ICE,Siegburg/Bonn,18.08.2016 07:08,18.08.2016 07:09,PROGNOSE,18.08.2016 07:10,18.08.2016 07:11,PROGNOSE,False


In [6]:
arr = (pd.to_datetime(data['ANKUNFTSZEIT']).astype(np.int64) - pd.to_datetime(data['AN_PROGNOSE']).astype(np.int64))/1000000000

In [8]:
dep = (pd.to_datetime(data['ABFAHRTSZEIT']).astype(np.int64) - pd.to_datetime(data['AB_PROGNOSE']).astype(np.int64))/1000000000

In [10]:
data10 = data

In [11]:
data10.drop('ANKUNFTSZEIT', axis=1, inplace=True)
data10.drop('AN_PROGNOSE', axis=1, inplace=True)
data10.drop('ABFAHRTSZEIT', axis=1, inplace=True)
data10.drop('AB_PROGNOSE', axis=1, inplace=True)
data10.drop('AN_PROGNOSE_STATUS', axis=1, inplace=True)
data10.drop('AB_PROGNOSE_STATUS', axis=1, inplace=True)
data10.drop('DURCHFAHRT_TF', axis=1, inplace=True)

In [12]:
data10.head()

,LINIEN_ID,LINIEN_TEXT,HALTESTELLEN_NAME
17,10057.0,IR,Thun
18,10057.0,IR,Spiez
23,101.0,ICE,Wuppertal Hbf
24,101.0,ICE,Köln Hbf
25,101.0,ICE,Siegburg/Bonn


In [13]:
from sklearn.preprocessing import OneHotEncoder 
from sklearn.preprocessing import LabelEncoder
enc = OneHotEncoder()
lbl_text = LabelEncoder()
enc1 = OneHotEncoder()
lbl_text1 = LabelEncoder()

In [34]:
lbl_text.fit(data10['LINIEN_TEXT'])
lbl_text1.fit(data10['HALTESTELLEN_NAME'])
enc.fit(lbl_text.transform(data10['LINIEN_TEXT']).reshape(-1,1))
enc1.fit(lbl_text1.transform(data10['HALTESTELLEN_NAME']).reshape(-1,1))

OneHotEncoder(categorical_features='all', dtype=<class 'float'>,
       handle_unknown='error', n_values='auto', sparse=True)

In [104]:
from sklearn.linear_model import SGDRegressor
arrivalReg = SGDRegressor(loss="huber")

In [105]:
X
for i in range(1,271):
    print(i)
    strat = (i-1) * 10000
    end = i * 10000
    if end > len(data10):
        end = len(data10)
    LINIEN_TEXT = pd.DataFrame(enc.transform(lbl_text.transform(data10.loc[range(strat, end)]['LINIEN_TEXT']).reshape(-1,1)).toarray())
    HALTESTELLEN_NAME = pd.DataFrame(enc1.transform(lbl_text1.transform(data10.loc[range(strat, end)]['HALTESTELLEN_NAME']).reshape(-1,1)).toarray())
    dd = data10.loc[range(strat, end)]
    dd.index = range(0, end-strat)
    X = pd.concat([dd, LINIEN_TEXT, HALTESTELLEN_NAME], axis=1)
    X.drop('HALTESTELLEN_NAME', axis=1, inplace=True)
    X.drop('LINIEN_TEXT', axis=1, inplace=True)
    arrivalReg.partial_fit(X, arr1.loc[range(strat, end)])
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270


In [137]:
import requests
import json
BASE_URL_CON = "http://transport.opendata.ch/v1/connections"
r = requests.get(BASE_URL_CON + '?' + "from=Lausanne&to=Berne&time=15:30")
response = json.loads(r.text)
for i in range(0, len(response['connections'][0]['sections'])):
    print(response['connections'][0]['sections'][i]['departure']['location']['name'])
    print(response['connections'][0]['sections'][i]['arrival']['location']['name'])
    print(response['connections'][0]['sections'][i]['journey']['category'])
    print(response['connections'][0]['sections'][i]['journey']['number'])
    LINIEN_TEXT = pd.DataFrame(enc.transform(lbl_text.transform(response['connections'][0]['sections'][i]['journey']['category']).reshape(-1,1)).toarray())
    HALTESTELLEN_NAME = pd.DataFrame(enc1.transform(lbl_text1.transform(response['connections'][0]['sections'][i]['arrival']['location']['name']).reshape(-1,1)).toarray())
    x_test = pd.concat([pd.DataFrame([response['connections'][0]['sections'][i]['journey']['number']]),LINIEN_TEXT,HALTESTELLEN_NAME], axis=1)
    #print(x_test)
    print(arrivalReg.predict(x_test))

Lausanne
Bern
IR
2527
[-464.16276112]


In [134]:
data10[data10['LINIEN_ID'] == 2]

,LINIEN_ID,LINIEN_TEXT,HALTESTELLEN_NAME


In [182]:
import math
def predit (start, end, hour, minutes):
    returnArr = []
    try:
        BASE_URL_CON = "http://transport.opendata.ch/v1/connections"
        r = requests.get(BASE_URL_CON + '?' + "from=" + start +"&to=" + end + "&time=" + str(hour) + ":" + str(minutes))
        response = json.loads(r.text)
        for i in range(0, len(response['connections'][0]['sections'])):
            #print(response['connections'][0]['sections'][i]['departure']['location']['name'])
            #print(response['connections'][0]['sections'][i]['arrival']['location']['name'])
            #print(response['connections'][0]['sections'][i]['journey']['category'])
            #print(response['connections'][0]['sections'][i]['journey']['number'])
            LINIEN_TEXT = pd.DataFrame(enc.transform(lbl_text.transform(response['connections'][0]['sections'][i]['journey']['category']).reshape(-1,1)).toarray())
            HALTESTELLEN_NAME = pd.DataFrame(enc1.transform(lbl_text1.transform(response['connections'][0]['sections'][i]['arrival']['location']['name']).reshape(-1,1)).toarray())
            x_test = pd.concat([pd.DataFrame([response['connections'][0]['sections'][i]['journey']['number']]),LINIEN_TEXT,HALTESTELLEN_NAME], axis=1)
            #print(x_test)
            time = math.floor(arrivalRegNew.predict(x_test)/60)
            #print(time)
            if (time < 0):
                time = time  * (-1)
                returnArr.append("Your train form " + response['connections'][0]['sections'][i]['departure']['location']['name'] + " to " + "is expected to arrive " + str(time) + " minutes late")
            elif (time > 0):
                returnArr.append("Your train form " + response['connections'][0]['sections'][i]['departure']['location']['name'] + " to " + "is expected to arrive " + str(time) + " minutes early")
            else:
                returnArr.append("Your train form " + response['connections'][0]['sections'][i]['departure']['location']['name'] + " to " + "is expected to arrive on time")
        return returnArr         
    except:
        returnArr.append("Ooops looks like we encourted an error")
        return returnArr

In [183]:
print(predit("Zurich", "Lugano", 15, 30 ))
#print(predit("Zurich", "Geneva", 15, 30 ))
#print(predit("Lausanne", "Basel", 15, 30 ))
print(predit("RSTDYGFTDTFYGUHIK", "zddfSedsdFsdfSDFD", 15, 30 ))

['Your train form Zürich HB to is expected to arrive 1 minutes late']
['Ooops looks like we encourted an error']


In [177]:
from sklearn.externals import joblib
joblib.dump(arrivalReg, 'model.pkl') 

['model.pkl', 'model.pkl_01.npy', 'model.pkl_02.npy']

In [178]:
arrivalRegNew = joblib.load('model.pkl')

In [179]:
arrivalRegNew.

SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='huber', n_iter=5, penalty='l2', power_t=0.25,
       random_state=None, shuffle=True, verbose=0, warm_start=False)